In [1]:
%pwd

'e:\\github_clone\\Text-Summarization-English\\research'

In [2]:
import os
os.chdir('../')

In [11]:
%pwd

'e:\\github_clone\\Text-Summarization-English'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from textSummerizerEnglish.constants import *
from textSummerizerEnglish.utils.common import create_directories, read_yaml

In [14]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [15]:
import urllib.request as request

In [21]:
import os
from rarfile import RarFile
import urllib.request as request
import zipfile
from textSummerizerEnglish.logging import logger
from textSummerizerEnglish.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-04 13:06:42,532: INFO: yaml file : config\config.yaml loaded successfully]
[2023-12-04 13:06:42,533: INFO: yaml file : params.yaml loaded successfully]
[2023-12-04 13:06:42,534: INFO: Created directory at artifacts]
[2023-12-04 13:06:42,535: INFO: Created directory at artifacts/data_ingestion]
[2023-12-04 13:06:46,841: INFO: artifacts/data_ingestion/news_summary.zip download! with following info: 
Connection: close
Content-Length: 4254236
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7d3f546dc385bc76c9b4804737e4ba9db8097592625e5c179df00c18ebdf3d1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3284:2E32DE:290F8D:34FBF3:656D818B
Accept-Ranges: bytes
Date: Mon, 04 Dec 2023 07:36:44 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1230-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1701675